In [1]:
import os
os.chdir("/home/jovyan")

In [2]:
import plotly
import plotly.express as px
import pandas as pd
import numpy as np

In [3]:
comment_df = pd.read_csv("data/processed/comments.csv")
comment_df.head()

,post_id,date,id,username,text,emoji,emoji_flag,pos,neg,neu,compound
0,2714737715189847981,2021-11-25 12:06:52,17920397863932145,chillhihi_715,好可愛❤️😍,❤😍,1,0.643,0.000,0.357,0.8020
1,2714737715189847981,2021-11-25 12:07:34,17949462172592069,shirley__0122,二伯很可愛😍😍,😍😍,1,0.500,0.000,0.500,0.7184
2,2714737715189847981,2021-11-25 12:07:38,17878235780597433,yje11082,二伯太美了😍🔥❤️,😍🔥❤,1,0.529,0.176,0.294,0.7003
3,2714737715189847981,2021-11-25 12:08:00,18101035180282801,wuyoyo0124,大都會公園欸👀,👀,1,0.000,0.000,1.000,0.0000
4,2714737715189847981,2021-11-25 12:09:33,18134899741245251,j016056,😂😂😂😂😂,😂😂😂😂😂,1,0.437,0.218,0.345,0.9260


In [4]:
post_df = pd.read_csv("data/processed/posts.csv")
post_df.head()

,kol_username,post_id,likes_count,comments_count,tags,time,videos,images
0,2uncle987,2714737715189847981,22855,27,['媽媽人生'],2021-11-25 12:06:16,0,6
1,2uncle987,2714735450530389672,15038,25,"['完整版影片在二伯的YouTube頻道喔', '限動有影片連結']",2021-11-25 12:02:29,1,0
2,2uncle987,2714704961886214739,18408,58,"['隨時打開新的我', 'DisneyPlus', '我要DisneyPlus帶我']",2021-11-25 11:01:11,0,4
3,2uncle987,2714036526479360974,41040,80,['超直男很難溝通耶哈哈哈'],2021-11-24 12:53:07,0,5
4,2uncle987,2713308846545097101,37847,20,['真的是好累不想要再一次了哈哈哈'],2021-11-23 12:47:21,0,1


In [5]:
profile_df = pd.read_csv("data/processed/profiles.csv")
profile_df.head()

,kol_username,followers_count,postss_count,external_url
0,2uncle987,786705,1855,https://www.hahababyselect.com/
1,1006nk,684126,2097,https://www.youtube.com/user/kyo0622ace1988
2,riceandshine.co,578032,2001,https://youtu.be/sDQJbnkjPi8
3,ju.lee.daily,412348,4649,http://line.me/S/shop/sticker/author/81411
4,Ilovesukilee,167784,1411,http://youtube.com/c/sukilee520


## How often 

## How often does each post gets emojis
Avg number of emoji per post

In [6]:
post_avg_emoji_df = comment_df.groupby(["post_id"])["emoji_flag"].agg([np.mean, np.sum]).reset_index()
post_avg_emoji_df.rename({"mean": "avg_num_emoji", "sum": "sum_num_emoji"}, axis=1, inplace=True)
post_avg_emoji_df.head()

,post_id,avg_num_emoji,sum_num_emoji
0,2689422774690065107,0.564103,44
1,2689910200653918753,0.760234,390
2,2690590577303457365,0.526316,10
3,2691294049090722860,0.833333,35
4,2692152131581897900,0.629630,68


Sentiment Distribution of each KOL

## Which KOL's followers prefer to user more emojis?

1. Average Emoji per KOL

In [44]:
post_avg_emoji_post_df = post_avg_emoji_df.merge(
    post_df[["kol_username", "post_id"]],
    on="post_id",
    how="left"
).groupby(["kol_username"])[["avg_num_emoji", "sum_num_emoji"]].mean().reset_index()
post_avg_emoji_post_df.sort_values(["avg_num_emoji"], ascending=False)

,kol_username,avg_num_emoji,sum_num_emoji
0,1006nk,0.696839,109.8
5,riceandshine.co,0.669175,251.5
6,sophieemomo,0.667185,14.6
1,2uncle987,0.665188,199.9
8,ycc_obliviate,0.634565,25.6
3,ju.lee.daily,0.610210,199.8
4,misscoolkiki,0.610079,315.9
7,tanyacowu,0.567366,26.0
2,ilovesukilee,0.554027,78.8


In [ ]:


fig = px.scatter(post_avg_emoji_post_df, x="gdpPercap", y="lifeExp",
	         size="pop", color="continent",
                 hover_name="country", log_x=True, size_max=60)
fig.show()


## Will more Videos or more Images lead to more emoji? (per post)

In [7]:
emoji_corr_df = post_avg_emoji_df.merge(
    post_df[["post_id", "videos", "images"]],
    how="left",
    on="post_id"
) 
emoji_corr_df.head()

,post_id,avg_num_emoji,sum_num_emoji,videos,images
0,2689422774690065107,0.564103,44,1,8
1,2689910200653918753,0.760234,390,1,0
2,2690590577303457365,0.526316,10,0,5
3,2691294049090722860,0.833333,35,0,1
4,2692152131581897900,0.629630,68,0,1


In [15]:
fig = px.scatter(emoji_corr_df, x="avg_num_emoji", y="images",
	         size="avg_num_emoji", size_max=10)
fig.show()

## Does more positive posts earn more likes?

In [18]:
emoji_likes_corr_df = post_avg_emoji_df.merge(
    post_df[["post_id", "likes_count", "kol_username"]],
    how="left",
    on="post_id"
) 
emoji_likes_corr_df.head()

,post_id,avg_num_emoji,sum_num_emoji,likes_count,kol_username
0,2689422774690065107,0.564103,44,18429,ilovesukilee
1,2689910200653918753,0.760234,390,25518,1006nk
2,2690590577303457365,0.526316,10,7884,1006nk
3,2691294049090722860,0.833333,35,22874,1006nk
4,2692152131581897900,0.629630,68,22116,1006nk


In [20]:
fig = px.scatter(emoji_likes_corr_df, x="likes_count", y="avg_num_emoji",
	         size="avg_num_emoji", size_max=30, log_x="likes_count")
fig.show()

## Which KOL is more positive

In [22]:
comment_df.head()

,post_id,date,id,username,text,emoji,emoji_flag,pos,neg,neu,compound
0,2714737715189847981,2021-11-25 12:06:52,17920397863932145,chillhihi_715,好可愛❤️😍,❤😍,1,0.643,0.000,0.357,0.8020
1,2714737715189847981,2021-11-25 12:07:34,17949462172592069,shirley__0122,二伯很可愛😍😍,😍😍,1,0.500,0.000,0.500,0.7184
2,2714737715189847981,2021-11-25 12:07:38,17878235780597433,yje11082,二伯太美了😍🔥❤️,😍🔥❤,1,0.529,0.176,0.294,0.7003
3,2714737715189847981,2021-11-25 12:08:00,18101035180282801,wuyoyo0124,大都會公園欸👀,👀,1,0.000,0.000,1.000,0.0000
4,2714737715189847981,2021-11-25 12:09:33,18134899741245251,j016056,😂😂😂😂😂,😂😂😂😂😂,1,0.437,0.218,0.345,0.9260


In [26]:
kol_positivity_df = comment_df[["pos", "neu", "neg", "post_id"]].merge(
    post_df[["post_id", "kol_username"]],
    how="left",
    on="post_id"
).groupby(["kol_username"])[["pos", "neu", "neg", "post_id"]].mean().reset_index()
kol_positivity_df

,kol_username,pos,neu,neg,post_id
0,1006nk,0.280317,0.383511,0.045023,2699467215285754368
1,2uncle987,0.270560,0.308492,0.065788,2712375333384205824
2,ilovesukilee,0.274586,0.279811,0.037194,2699182034825895424
3,ju.lee.daily,0.308403,0.281204,0.035161,2707795919455004160
4,misscoolkiki,0.219772,0.249125,0.071655,2704935745021407744
5,riceandshine.co,0.255298,0.355429,0.103157,2711456431262025728
6,sophieemomo,0.062759,0.079041,0.023921,2712652160657107456
7,tanyacowu,0.164807,0.218396,0.032136,2703675629490227200
8,ycc_obliviate,0.256119,0.317916,0.058067,2708897456752363520


In [36]:
import plotly.graph_objects as go


fig = go.Figure(data=[
    go.Bar(name='Positive Sentiment', x=kol_positivity_df["kol_username"], y=kol_positivity_df["pos"]),
    go.Bar(name='neu', x=kol_positivity_df["kol_username"], y=kol_positivity_df["neu"]),
    go.Bar(name='neg', x=kol_positivity_df["kol_username"], y=kol_positivity_df["neg"])
])
# Change the bar mode
fig.update_layout(barmode='group', title="Overall Sentiments of each KOL", title_xanchor="left", title_font_size=18)
fig.show()

## Sentiment change through time of each KOL (not complete)

In [54]:
comment_df2 = comment_df.copy()
comment_df2.loc[:, "date"] = comment_df2["date"].apply(lambda x: pd.to_datetime(x, format="%Y-%m-%d")).dt.date
comment_df2.head()

,post_id,date,id,username,text,emoji,emoji_flag,pos,neg,neu,compound
0,2714737715189847981,2021-11-25,17920397863932145,chillhihi_715,好可愛❤️😍,❤😍,1,0.643,0.000,0.357,0.8020
1,2714737715189847981,2021-11-25,17949462172592069,shirley__0122,二伯很可愛😍😍,😍😍,1,0.500,0.000,0.500,0.7184
2,2714737715189847981,2021-11-25,17878235780597433,yje11082,二伯太美了😍🔥❤️,😍🔥❤,1,0.529,0.176,0.294,0.7003
3,2714737715189847981,2021-11-25,18101035180282801,wuyoyo0124,大都會公園欸👀,👀,1,0.000,0.000,1.000,0.0000
4,2714737715189847981,2021-11-25,18134899741245251,j016056,😂😂😂😂😂,😂😂😂😂😂,1,0.437,0.218,0.345,0.9260


In [53]:
comment_df2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20458 entries, 0 to 20457
Data columns (total 11 columns):
 #   Column      Non-Null Count  Dtype         
---  ------      --------------  -----         
 0   post_id     20458 non-null  int64         
 1   date        20458 non-null  datetime64[ns]
 2   id          20458 non-null  int64         
 3   username    20458 non-null  object        
 4   text        20458 non-null  object        
 5   emoji       12219 non-null  object        
 6   emoji_flag  20458 non-null  int64         
 7   pos         20458 non-null  float64       
 8   neg         20458 non-null  float64       
 9   neu         20458 non-null  float64       
 10  compound    20458 non-null  float64       
dtypes: datetime64[ns](1), float64(4), int64(3), object(3)
memory usage: 1.7+ MB
